# Projeto 1 - Ciência dos Dados

Nome: Gustavo Oliveira

Nome: Gabriela Duarte

Nome: Vinicius Morales

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\viniz\GitRepositories\Projeto-1-CDados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'trabalho1bbb.gu.xlsx'

In [4]:
train = pd.read_excel(filename)
filtra_linhas1 = train['BBB 21'] == 'x'
filtra_linhas2 = train['Outros BBB'] == 'x'
filtra_linhas3 = train['irrelevantes'] == 'x'
treinamento = train.loc[:, ['Treinamento', 'BBB 21', 'Outros BBB', 'irrelevantes']]
treinamento.head(5)

,Treinamento,BBB 21,Outros BBB,irrelevantes
0,"flans, corre aqui! @gecychaves @meninodelll @m...",NaN,NaN,x
1,@m0m0v0gue sim \n\n#bbtvi2020 #followtrick #fo...,NaN,NaN,x
2,so conheço o 1 e 3 ent nao posso opinar\n\n#fo...,NaN,NaN,x
3,vou fazer meu msd de uma vez\n\n#follotrick #b...,NaN,NaN,x
4,legal as minhas favs 😍\nmas acho que vou de an...,NaN,NaN,x


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevancia,BBB 21,Outros BBB,fiuk,paredão,irrelevantes
0,meu detox do twitter é ler os comentários da p...,0,NaN,x,NaN,NaN,NaN
1,"de acordo com o site notícias da tv, a #globo ...",0,NaN,NaN,NaN,NaN,x
2,@webtvbrasileira\n\nassista #bigbrotherbrasil ...,0,NaN,NaN,NaN,NaN,x
3,@jkprities sim \n\n#bbtvi2020 #followtrick #fo...,0,NaN,NaN,NaN,NaN,x
4,"""quem quer fica. quem não quer vai embora.""\n ...",0,x,NaN,NaN,x,NaN


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Nossos dados são sobre o BBB, onde nós filtramos eles por 3 categorias, se o tweet é relacionado a BBB 21, outros BBBs e se o Tweet é relevante ou não.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
import re


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '\@\w+\s?|[\n!-,.:?;|]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    
    return text_subbed

for i in range(500):
    train.Treinamento[i] = cleanup(train.Treinamento[i].lower())

train

<ipython-input-6-41a49bd78ff1>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.Treinamento[i] = cleanup(train.Treinamento[i].lower())


,Treinamento,Relevancia,BBB 21,Outros BBB,fiuk,paredão,irrelevantes
0,flans corre aqui a quixitagem ta fofona dema...,0,NaN,NaN,NaN,NaN,x
1,sim bbtvi2020 followtrick follotrick kpop kpop...,0,NaN,NaN,NaN,NaN,x
2,so conheço o 1 e 3 ent nao posso opinarfollotr...,0,NaN,NaN,NaN,NaN,x
3,vou fazer meu msd de uma vezfollotrick bbtvi r...,0,NaN,NaN,NaN,NaN,x
4,legal as minhas favs 😍mas acho que vou de anyf...,0,NaN,NaN,NaN,NaN,x
...,...,...,...,...,...,...,...
495,rt sigam nosso instagram e fiquem por dentro ...,1,NaN,NaN,NaN,x,NaN
496,terminei de ver o episódio s21 e52 de big bro...,1,NaN,x,NaN,NaN,NaN
497,rt ai queria ficar aqui tweetando sobre a fes...,1,x,NaN,NaN,NaN,NaN
498,bbb tá on gizelly bicalho lembra momentos do b...,1,NaN,x,NaN,NaN,NaN


In [7]:
filtra_linhas1 = train['BBB 21'] == 'x'
tabelaBBB21 = train.loc[filtra_linhas1,['Treinamento']] # Tabela filtrada com apenas os dados relevantes de BBB21

filtra_linhas2 = train['Outros BBB'] == 'x'
tabela_outros = train.loc[filtra_linhas2,['Treinamento']] # Tabela filtrada com apenas os dados relevantes de Outros BBB

filtra_linhas3 = train['irrelevantes'] == 'x'
tabela_ir = train.loc[filtra_linhas3,['Treinamento']] # Tabela filtrada com apenas os dados relevantes de Irrelevantes

In [8]:
# Transformando as linhas dos DataFrames em arrays de strings
palavras_train_BBB21 = tabelaBBB21.values.tolist()
palavras_train_outros = tabela_outros.values.tolist()
palavras_train_ir = tabela_ir.values.tolist()

In [9]:
# Fazendo o join dos arrays em strings
palavras_train_BBB21 = ''.join(x for sub in palavras_train_BBB21 for x in sub)
palavras_train_outros = ''.join(x for sub in palavras_train_outros for x in sub)
palavras_train_ir = ''.join(x for sub in palavras_train_ir for x in sub)

In [10]:
palavras_train_BBB21 = palavras_train_BBB21.split()
serieBBB21 = pd.Series(palavras_train_BBB21)
tabela_relativa_BBB21 = serieBBB21.value_counts(True)
tabela_relativa_BBB21.head

<bound method NDFrame.head of bbb21                         0.036020
bigbrotherbrasil              0.031957
e                             0.027784
a                             0.022622
o                             0.022293
                                ...   
funciona                      0.000110
brep                          0.000110
diazhttps//tco/pomrql9demo    0.000110
adoooooro                     0.000110
forasarahvihh                 0.000110
Length: 2396, dtype: float64>

In [11]:
palavras_train_outros = palavras_train_outros.split()
serie_outros = pd.Series(palavras_train_outros)
tabela_relativa_outros = serie_outros.value_counts(True)
tabela_relativa_outros.head()

bigbrotherbrasil    0.048689
de                  0.044944
brasil              0.037453
brother             0.037453
s21                 0.037453
dtype: float64

In [12]:
palavras_train_ir = palavras_train_ir.split()
serie_ir = pd.Series(palavras_train_ir)
tabela_relativa_ir = serie_ir.value_counts(True)
tabela_relativa_ir.head()

bbb21               0.061669
bigbrotherbrasil    0.039903
kpop                0.035067
redebbb             0.031439
kpoptwitter         0.025393
dtype: float64

In [13]:
tabela_relativa_BBB21.sum()

1.0

In [14]:
tabela_relativa_outros.sum()

0.9999999999999999

In [15]:
tabela_relativa_ir.sum()

0.9999999999999998

In [75]:
tabela_relativa_BBB21

bbb21                         0.036020
bigbrotherbrasil              0.031957
e                             0.027784
a                             0.022622
o                             0.022293
                                ...   
funciona                      0.000110
brep                          0.000110
diazhttps//tco/pomrql9demo    0.000110
adoooooro                     0.000110
forasarahvihh                 0.000110
Length: 2396, dtype: float64

In [76]:
tabela_relativa_outros

bigbrotherbrasil         0.048689
de                       0.044944
brasil                   0.037453
brother                  0.037453
s21                      0.037453
                           ...   
afazenda                 0.003745
uma                      0.003745
https//tco/amhmu6tud8    0.003745
e                        0.003745
ver                      0.003745
Length: 148, dtype: float64

In [77]:
tabela_relativa_ir

bbb21                        0.061669
bigbrotherbrasil             0.039903
kpop                         0.035067
redebbb                      0.031439
kpoptwitter                  0.025393
                               ...   
globoplay                    0.001209
bigbrotherbrasil21           0.001209
gaeldisouza                  0.001209
teamjuliettefreiretadinha    0.001209
foraarthur                   0.001209
Length: 371, dtype: float64

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [70]:
tabela_relativa_BBB21['bbb']

0.012738853503184714

In [71]:
tabela_relativa_outros['bbb']

0.00749063670411985

In [72]:
tabela_relativa_ir['bbb']

0.014510278113663845

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**